# 0.0 Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D


2024-11-10 14:58:49.016804: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 14:58:49.018245: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 14:58:49.054264: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 14:58:49.055517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 14:58:49.625108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
data = pd.read_csv('Tweets.csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
data['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [4]:
tweets = data[data['airline_sentiment_confidence'] > 0.8]

In [5]:
token = Tokenizer( num_words=100 )
token.fit_on_texts(tweets['text'].values)

In [6]:
X = token.texts_to_sequences( tweets['text'].values )
X = pad_sequences( X, padding='post', maxlen=100 )

In [7]:
label_enc = LabelEncoder()
y = label_enc.fit_transform( tweets['airline_sentiment'] )

In [8]:
y = to_categorical( y )
print( y )

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [9]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3 )

In [10]:
model = Sequential()
model.add( Embedding( input_dim=len( token.word_index ), output_dim=128, input_length=X.shape[1] ) )
model.add( SpatialDropout1D( 0.2 ) )
model.add( LSTM( units=196, dropout=0.2, recurrent_dropout=0.2 ) )
model.add( Dense( units=3, activation='softmax' ) )

2024-11-10 14:58:50.936345: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-10 14:58:50.936643: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1638656   
                                                                 
 spatial_dropout1d (Spatial  (None, 100, 128)          0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 1894047 (7.23 MB)
Trainable params: 1894047 (7.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [12]:
model.fit( X_train, y_train, epochs=10, batch_size=30, verbose=True, validation_data=(X_test, y_test) )

Epoch 1/10
245/245 [==============================] - 26s 98ms/step - loss: 0.8243 - accuracy: 0.7036 - val_loss: 0.8099 - val_accuracy: 0.7091
Epoch 2/10
245/245 [==============================] - 27s 112ms/step - loss: 0.8149 - accuracy: 0.7058 - val_loss: 0.8057 - val_accuracy: 0.7091
Epoch 3/10
245/245 [==============================] - 33s 133ms/step - loss: 0.8140 - accuracy: 0.7058 - val_loss: 0.8046 - val_accuracy: 0.7091
Epoch 4/10
245/245 [==============================] - 34s 139ms/step - loss: 0.8129 - accuracy: 0.7058 - val_loss: 0.8122 - val_accuracy: 0.7091
Epoch 5/10
245/245 [==============================] - 31s 127ms/step - loss: 0.8128 - accuracy: 0.7058 - val_loss: 0.8066 - val_accuracy: 0.7091
Epoch 6/10
245/245 [==============================] - 25s 101ms/step - loss: 0.8124 - accuracy: 0.7058 - val_loss: 0.8069 - val_accuracy: 0.7091
Epoch 7/10
245/245 [==============================] - 26s 106ms/step - loss: 0.8121 - accuracy: 0.7058 - val_loss: 0.8053 - val_acc

In [13]:
loss, accuracy = model.evaluate( X_test, y_test )
print( 'Loss:', loss )
print( 'Accuracy:', accuracy )

99/99 [==============================] - 3s 31ms/step - loss: 0.8063 - accuracy: 0.7091
Loss: 0.8062670230865479
Accuracy: 0.7090503573417664


In [14]:
prev = model.predict( X_test )
print( prev )

99/99 [==============================] - 3s 28ms/step
[[0.68307805 0.15832064 0.15860134]
 [0.68307805 0.15832064 0.15860134]
 [0.68307805 0.15832064 0.15860134]
 ...
 [0.68307805 0.15832064 0.15860137]
 [0.68307805 0.15832064 0.15860137]
 [0.68307805 0.15832064 0.15860137]]
